In [11]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from imgaug import augmenters as iaa
import imgaug as ia
import os
from filemanager import get_file_paths as gfp
import matplotlib.font_manager as fm

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

IMAGE_WIDTH = 256
IMAGE_HEIGHT = 256
IMAGE_CHANNEL = 3
NUM_IMAGE = 10000

NUM_CLASS = 50

tf.random.set_seed(42)
ia.seed(42)

# 레이블 불러오기

In [12]:
dataset = gfp.File_Control.searchAllFilesInDirectoryByDir("/raid/food_competition_KT_set1/train", "jpg")
dataset.sort()
label_dict = {}
for i in range(len(dataset)):
    label = os.path.dirname(dataset[i][0])
    label = os.path.basename(label)
    label_dict[str(i)] = label
print(label_dict)

{'0': '가자미전', '1': '간장게장', '2': '감자탕', '3': '거봉포도', '4': '고구마', '5': '고구마맛탕', '6': '고등어찌개', '7': '곱창구이', '8': '군만두', '9': '굴전', '10': '김치찌개', '11': '깻잎나물볶음', '12': '꼬리곰탕', '13': '꽈리고추무침', '14': '나시고랭', '15': '누룽지', '16': '단무지', '17': '달걀말이', '18': '달걀볶음밥', '19': '달걀비빔밥', '20': '닭가슴살', '21': '닭개장', '22': '닭살채소볶음', '23': '닭칼국수', '24': '도가니탕', '25': '도토리묵', '26': '돼지감자', '27': '돼지고기구이', '28': '두부', '29': '두부고추장조림', '30': '딸기', '31': '떡갈비', '32': '떡국', '33': '레드와인', '34': '마늘쫑무침', '35': '마카롱', '36': '매운탕', '37': '미소된장국', '38': '미소장국', '39': '미역초무침', '40': '바나나우유', '41': '바지락조개국', '42': '보리밥', '43': '불고기', '44': '비빔밥', '45': '뼈해장국', '46': '삼선자장면', '47': '새우매운탕', '48': '새우볶음밥', '49': '생연어'}


# 기존 데이터 불러오기

In [13]:
x = np.load('/raid/food_competition_KT_set1/kfood_img.npy')
y = np.load('/raid/food_competition_KT_set1/kfood_y.npy')

np.random.seed(0)
np.random.shuffle(x)
np.random.seed(0)
np.random.shuffle(y)

# Augmentation

In [4]:
rand_aug = iaa.RandAugment(n=3, m=7)

def augment(images):
    # Input to `augment()` is a TensorFlow tensor which
    # is not supported by `imgaug`. This is why we first
    # convert it to its `numpy` variant.
    images = tf.cast(images, tf.uint8)
    return rand_aug(images=images.numpy())

# 모델 학습

In [5]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.resnet_v2 import ResNet50V2

from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

In [6]:
def resnet_model(img_height, img_width, num_classes):
    model = ResNet50V2(
        include_top=True,
        input_shape=(img_width, img_height, 3),
        weights=None,
        classes=50,
    )
    return model

model = resnet_model(IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CLASS)
model.summary()

2022-09-17 05:22:08.922647: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-17 05:22:11.979429: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21206 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:23:00.0, compute capability: 8.0


Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                         

                                                                                                  
 conv2_block3_preact_relu (Acti  (None, 64, 64, 256)  0          ['conv2_block3_preact_bn[0][0]'] 
 vation)                                                                                          
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 64, 64, 64)   16384       ['conv2_block3_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv2_block3_1_bn (BatchNormal  (None, 64, 64, 64)  256         ['conv2_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_blo

 conv3_block2_2_conv (Conv2D)   (None, 32, 32, 128)  147456      ['conv3_block2_2_pad[0][0]']     
                                                                                                  
 conv3_block2_2_bn (BatchNormal  (None, 32, 32, 128)  512        ['conv3_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block2_2_relu (Activatio  (None, 32, 32, 128)  0          ['conv3_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_3_conv (Conv2D)   (None, 32, 32, 512)  66048       ['conv3_block2_2_relu[0][0]']    
                                                                                                  
 conv3_blo

 conv4_block1_1_conv (Conv2D)   (None, 16, 16, 256)  131072      ['conv4_block1_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv4_block1_1_bn (BatchNormal  (None, 16, 16, 256)  1024       ['conv4_block1_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block1_1_relu (Activatio  (None, 16, 16, 256)  0          ['conv4_block1_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block1_2_pad (ZeroPaddin  (None, 18, 18, 256)  0          ['conv4_block1_1_relu[0][0]']    
 g2D)     

                                                                                                  
 conv4_block3_2_relu (Activatio  (None, 16, 16, 256)  0          ['conv4_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block3_3_conv (Conv2D)   (None, 16, 16, 1024  263168      ['conv4_block3_2_relu[0][0]']    
                                )                                                                 
                                                                                                  
 conv4_block3_out (Add)         (None, 16, 16, 1024  0           ['conv4_block2_out[0][0]',       
                                )                                 'conv4_block3_3_conv[0][0]']    
                                                                                                  
 conv4_blo

 conv4_block6_1_bn (BatchNormal  (None, 16, 16, 256)  1024       ['conv4_block6_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block6_1_relu (Activatio  (None, 16, 16, 256)  0          ['conv4_block6_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block6_2_pad (ZeroPaddin  (None, 18, 18, 256)  0          ['conv4_block6_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv4_block6_2_conv (Conv2D)   (None, 8, 8, 256)    589824      ['conv4_block6_2_pad[0][0]']     
          

                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 8, 8, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_out (Add)         (None, 8, 8, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_conv[0][0]']    
                                                                                                  
 conv5_block3_preact_bn (BatchN  (None, 8, 8, 2048)  8192        ['conv5_block2_out[0][0]']       
 ormalization)                                                                                    
                                                                                                  
 conv5_block3_preact_relu (Acti  (None, 8, 8, 2048)  0           ['conv5_block3_preact_bn[0][0]'] 
 vation)  

In [7]:
path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
prop = fm.FontProperties(fname=path, size=18)

In [18]:
#dataset = read_dataset(100, './dataset', 'kt_food')

NUM_FOLD = 5
EPOCHS = 30
BATCH_SIZE = 32
AUTO = tf.data.AUTOTUNE

kfold = KFold(n_splits=NUM_FOLD, shuffle=True, random_state=30)

fold_no = 1
results = []

for train_index, test_index in kfold.split(x):
    
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    x_train = augment(x_train)
    
    print("test index : ", test_index)
    print("test index shape : ", test_index.shape)
    
    model = resnet_model(IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CLASS)
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    print('-' * 50)
    print(f'Training for fold {fold_no} ...')
    model.fit(x=x_train, y=y_train, validation_data=(x_test, y_test), epochs=EPOCHS)
    eval_results = model.evaluate(x_test, y_test)
    print("test loss, test acc : ", eval_results)
    results.append(eval_results)
    fold_no = fold_no + 1
    
    break
    


test index :  [   7   12   13 ... 9987 9991 9993]
test index shape :  (2000,)
--------------------------------------------------
Training for fold 1 ...
Epoch 1/30
250/250 [==============================] - 55s 209ms/step - loss: 3.4750 - accuracy: 0.1101 - val_loss: 42.1849 - val_accuracy: 0.0405
Epoch 2/30
250/250 [==============================] - 44s 175ms/step - loss: 3.0544 - accuracy: 0.1651 - val_loss: 6.4453 - val_accuracy: 0.1185
Epoch 3/30
250/250 [==============================] - 42s 168ms/step - loss: 2.8967 - accuracy: 0.2155 - val_loss: 3.4207 - val_accuracy: 0.1960
Epoch 4/30
250/250 [==============================] - 50s 200ms/step - loss: 2.7164 - accuracy: 0.2419 - val_loss: 3.6675 - val_accuracy: 0.1755
Epoch 5/30
250/250 [==============================] - 51s 204ms/step - loss: 2.6052 - accuracy: 0.2743 - val_loss: 2.2000 - val_accuracy: 0.3705
Epoch 6/30
250/250 [==============================] - 52s 207ms/step - loss: 2.4659 - accuracy: 0.3104 - val_loss: 2.4402